
# Introduction

This past week a paper came out from a combination of the QIIME developers and the developers of some recent OTU clustering methods: [Open-Source Sequence Clustering Methods Improve the State Of the Art](http://msystems.asm.org/content/1/1/e00003-15). In this paper, several new and open source methods (Swarm, SUMACLUST, OTUCLUST) are compared to the standard methods in the field (QIIME/uclust, mothur, UPARSE).

As the developer of [an open source method for inferring sample composition from amplicon sequencing](http://biorxiv.org/content/early/2015/08/06/024034), this presents an ideal opportunity to compare the accuracy our method, DADA2, to the "state of the art".

To briefly recap, DADA2 is a method to infer the sample sequences in a collection of amplicon sequencing reads. DADA2 is a de novo method, and completely reference free. DADA2 does not create OTUs, it infers sample sequences exactly, resolving differences of as little as one nucleotide. DADA2 is an open-source method implemented as [an R package available here](http://http://benjjneb.github.io/dada2/).

This document is the compiled version of an R markdown document ([available here](https://www.dropbox.com/sh/atmpqqm9zxwf6qf/AAB9MPLGTh6ZWWBAV1HSWNoca?dl=0)) and all the code here can be replicated on your own computer.

**Load packages and set working directory**:


In [ ]:
library(ShortRead); packageVersion("ShortRead")
library(dada2); packageVersion("dada2")
setwd("~/Desktop/OpenOTU/")


*To run this R code yourself, [install the DADA2 R package](http://http://benjjneb.github.io/dada2/), download [the supplementary data from the paper](ftp://ftp.microbio.me/pub/supplemental_otu_clustering_datasets.tar.gz), unzip it, and set your R session's working directory to the folder in which it was unzipped.*

# Simulated data

The first datasets considered in the paper are simulated datasets from a known mixture of template sequences. Errors are added using a software program that attempts to mimic the errors introduced by PCR amplification and Illumina sequencing. The GreenGenes code for the correct bacterial strain is included in the id line of the fastq file, thus we know the correct mapping for every read in these datasets.

## Even abundances dataset

The first dataset considered in the paper is a simulated dataset, consisting of an even distribution (100 reads each) of 1,076 randomly selected GreenGenes 16S V4 gene segments.

This is basically the easiest possible problem for these methods: There are no chimeras, errors are simulated and won't include pathological cases, and the distribution of abundances is completely even. The one difficulty is that these strains are separated from each other by as little as one nucleotide in some cases.

### Removing records with ambiguous nucleotides

When generating this dataset, the authors left in several sequence templates from GreenGenes that included ambiguous nucleotides (eg. K, Y...) in the V4 region. This results in sequencing reads containing those ambiguous bases in the fastq files, which would never actually occur in Illumina sequencing. So we're going to start by removing those records:



In [ ]:
getGGIDs <- function(fn) { # Extracts the GG IDs from the provided fastq file
  srq <- readFastq(fn)
  ids <- sapply(strsplit(as.character(id(srq)), "\\s"), `[`, 2)
  ids <- sapply(strsplit(ids, "-"), `[`, 1)
  ids
}

fn.even <- "supplemental_otu_clustering_datasets/UPARSE_not_filtered_QIIME_label_format/16S/even/seqs.fastq"
ids <- getGGIDs(fn.even)
srq <- readFastq(fn.even)
seqs <- as.character(sread(srq))
no.ambig <- sapply(seqs, dada2:::C_check_ACGT)
table(ids[!no.ambig])


There are 9 variants with ambiguous bases in the V4 region being used as templates. We remove them:


In [ ]:
ambig.ids <- names(table(ids[!no.ambig]))
fn.even.noN <- "even.noN.fastq.gz"
if(file.exists(fn.even.noN)) file.remove(fn.even.noN)
writeFastq(srq[!ids %in% ambig.ids], fn.even.noN, compress=TRUE, width=20000L)


There are now **1,067** green-genes strains in the dataset, each with 100 reads.

### Running DADA2

This is simulated data, but its always good practice to look at the quality profile of the data before jumping in:


In [ ]:
qq <- qa(fn.even.noN)[["perCycle"]]$quality
ShortRead:::.plotCycleQuality(qq, main="Sim - Even")


It looks very simulated. Typically we recommend trimming the first 10 bases and perhaps some at the end, as well as quality filtering based on expected errors. But in the interests of getting nice round numbers at the end, we're going filter-free!

Entering the DADA2 pipeline (minus filtering and chimera-checking):


In [ ]:
derep.even <- derepFastq(fn.even.noN, verbose=TRUE)
dada.even <- dada(derep.even, err=inflateErr(tperr1, 3), selfConsist=TRUE)
dada.even


The DADA2 pipeline inferred **1,055** sample sequences. This is more than any of the other algorithms detected (see Table 2 in the paper), but it's less than the 1,067 strains that made up this data.

### Evaluating DADA2's output

Let's take a look at the abundances of the inferred sample sequences.


In [ ]:
table(dada.even$clustering$abundance)


First off, we see that we are getting the abundance estimates essentially exactly right. 1,037/1,055 inferred sample sequences have an inferred abundance of 100, exactly what is expected. And this table also tells us why we aren't getting 1,067 variants: Several strains have identical 16S sequences over this region. There are 10 pairs of identical strains (the 200 abundance sequences) and 1 triple of identical strains (300 abundance). After subtracting out the 12 strains with duplicate sequences, we have **1,055** unique sequence variants in the dataset, **all** of which were detected by DADA2.

We can dig deeper by using the GreenGenes ID in the id line of of the fastq file to check whether the reads of each strain are all being assigned to the same output sequence.



In [ ]:
assigned.even <- dada.even$map[derep.even$map]
ggids.even <- getGGIDs(fn.even.noN)
joint.table <- table(ggids.even, assigned.even)
joint.mat <- as.matrix(joint.table)
# Rows are GreenGenes IDs from the fastq id line
# Columns are the number of the output sequence to which the read was assigned
# Entries are the number of reads with that ggID and assignment
table(rowSums(joint.mat>0))


The assignment is almost perfect, in 1,063/1,067 cases, all reads with a given ID are assigned to the same output sequence by DADA2. We look more closely at the 4 remaining cases:



In [ ]:
multi.assigned <- rowSums(joint.mat>0) > 1
joint.mat[multi.assigned, colSums(joint.mat[multi.assigned,]>0)>0]


Even these assignments are nearly perfect. In the first case 2 reads of GG strain 178767 (out of 100) are assigned to a different output sequence than the rest. In the other 3 cases just 1 read (out of 100) is differently assigned.

## Staggered abundances dataset

This second dataset is simulated from the same set of GreenGenes 16S sequence templates, but now the abundances of the strains are allowed to vary in abundance from 1 to 197 reads.

We start by again removing the problematic sequence variants with ambiguous nucleotides:


In [ ]:
fn.stag <- "supplemental_otu_clustering_datasets/UPARSE_not_filtered_QIIME_label_format/16S/staggered/seqs.fastq"
ids <- getGGIDs(fn.stag)
srq <- readFastq(fn.stag)

fn.stag.noN <- "stag.noN.fastq.gz"
if(file.exists(fn.stag.noN)) file.remove(fn.stag.noN)
writeFastq(srq[!ids %in% ambig.ids], fn.stag.noN, compress=TRUE, width=20000L)


As before, there are now 1,067 green-genes strains in the dataset, representing 1,055 unique sequence variants.

### Running DADA2

Entering the DADA2 pipeline (minus filtering and chimera-checking):


In [ ]:
derep.stag <- derepFastq(fn.stag.noN, verbose=TRUE)
dada.stag <- dada(derep.stag, err=inflateErr(tperr1, 3), selfConsist=TRUE)
dada.stag


DADA2 only infers **1,042** output sequences this time (of the 1,055 unique variants in the dataset). This is higher than any of the methods tested in the paper, in part because the high-resolution of DADA2 allows it to differentiate very similar sequence variants.

### Evaluating DADA2's output

Let's take a closer look at what was missed by using the fact that we know all the sequence variants from analyzing the even dataset. First we extract the GreenGenes IDs of the strains DADA2 missed in the staggered dataset:


In [ ]:
all(dada.stag$clustering$sequence %in% dada.even$clustering$sequence)
in.even.not.staggered <- which(!dada.even$clustering$sequence %in% dada.stag$clustering$sequence)
strain.lookup <- joint.mat[,in.even.not.staggered]
strain.lookup <- strain.lookup[rowSums(strain.lookup)>0,]
# strain.lookup # 1 strain per output sequence for these missed variants
missing.strains <- rownames(strain.lookup)
missing.strains


With the GGIDs of the missing strains in hand, let's look at their presence in the staggered dataset:


In [ ]:
ids.stag <- getGGIDs(fn.stag.noN)
table(ids.stag)[missing.strains]


There are 13 strains missed in the staggered dataset. Eight of the missing strains contribute only one read to the dataset. DADA2 does not call singleton variants (and singleton OTUs were excluded in the paper) because of how hard it is to identify real biological singletons from errors. The remaining missed strains had just 3 or 5 reads.

While we don't get easily interpretable round abundances here, we can still evaluate the accuracy of DADA2's inferred abundances overall:


In [ ]:
# Map between staggered assignments and even assignments by matching sequences
even.assign.of.stag.assign <- match(dada.stag$sequence, dada.even$sequence)
# Get the GGids of the unique variants corresponding to each corresponding even assignment
# This uses our knowledge that even assignments of >2 reads were all correct
ggids.of.corresponding.unique.variants <- lapply(even.assign.of.stag.assign, function(col) rownames(joint.mat)[(joint.mat>80)[,col]])
# Calculate the abundances of each unique variant from the fastq ID field
unique.variant.abundances <- sapply(ggids.of.corresponding.unique.variants, function(ids) sum(table(ids.stag)[ids]))
# Visualize
plot(dada.stag$clustering$abundance, unique.variant.abundances, xlab="DADA2 inferred abundance", ylab="True abundance")
abline(0,1,col="red")
cor(dada.stag$clustering$abundance, unique.variant.abundances)


DADA2's inferred abundances are extremely accurate.

# Bokulich Mock Communities

The second set of test datasets used in the paper are set of mock communities (i.e. mixtures of known strains) that were sequenced for the Bokulich et al. paper [Quality-filtering vastly improves diversity estimates from Illumina amplicon sequencing](http://dx.doi.org/10.1038/nmeth.2276).

## Mock Community 1688

This mock community dataset (also Bokulich_6 in the paper) is a single sample of a mixture of 48 strains. It is the smallest Bokulich dataset, at just 251k 150nt reads sequences on a Miseq, but is also the most diverse.

### Removing PhiX

It is not clear whether the phiX contamination in the Bokulich datasets were removed in the benchmarking paper, but we do so here regardless. For the unfamiliar, when performing Illumina amplicon sequencing, phiX DNA is added in order to maintain enough heterogeneity in base composition for the base-calling software to work. While the phiX DNA is often removed by the sequencing center, it is not uncommon for phiX sequences to remain, and you should always check for phiX before processing new amplicon sequencing data.

There are various ways to remove phiX contamination.  My solution is to locally BLAST against the phiX genome, and remove any read which hit the phiX genome with a custom python script. My shell workflow is below. It requires a [local installation of blast](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastDocs&DOC_TYPE=Download), the [phix genome](http://www.ncbi.nlm.nih.gov/nuccore/9626372) made into a blastdb, and [this simple python script](https://www.dropbox.com/sh/atmpqqm9zxwf6qf/AAB9MPLGTh6ZWWBAV1HSWNoca?dl=0). These commands should be executed in the supplemental\_otu\_clustering\_datasets/UPARSE\_not\_filtered\_QIIME\_label\_format/16S/1688/ directory:


In [ ]:
####### TO BE EXECUTED BY HAND IN YOUR SHELL
### awk '{ if (NR % 4 == 1) print ">",((NR+3)/4); else if (NR % 4 == 2) print $0; }' seqs.fastq > seqs.fasta
### blastn -query seqs.fasta -db phix -out phix_hits.out -max_target_seqs 1 -outfmt "6 qacc evalue"
### python rm_hits.py phix_hits.out seqs.fastq 1688_seqs_nophix.fastq


This reduces the number of sequencing reads from 250,903 to 247,244. *You can also [download the phiX free fastq file here](https://www.dropbox.com/sh/atmpqqm9zxwf6qf/AAB9MPLGTh6ZWWBAV1HSWNoca?dl=0)*.

### Quality Filtering

There are general guidelines for quality filtering (for example, we generally recommending filtering out reads with more than 2 expected errors) but one should always look at your data and modify those guidelines as necessary.

Visualizing the quality profile:


In [ ]:
fn.1688 <- "supplemental_otu_clustering_datasets/UPARSE_not_filtered_QIIME_label_format/16S/1688/1688_seqs_nophix.fastq"
qq <- qa(fn.1688)[["perCycle"]]$quality
ShortRead:::.plotCycleQuality(qq, main="Bokulich 1688")


The quality really falls off a cliff over the last 10 nucleotides. We'll trim those, and perform our standard recommended filters on expected errors and Ns:


In [ ]:
filt.1688 <- "filt_1688.fastq.gz"
if(file.exists(filt.1688)) file.remove(filt.1688)
fastqFilter(fn.1688, filt.1688, truncLen=140, truncQ=2, maxEE=2, maxN=0, verbose=TRUE)


### Running DADA2

Executing the DADA2 pipeline (with chimera-checking this time):


In [ ]:
derep.1688 <- derepFastq(filt.1688, verbose=TRUE)
dada.1688 <- dada(derep.1688, err=inflateErr(tperr1, 3), selfConsist=TRUE)
bim.1688 <- isBimeraDenovo(dada.1688, verbose=TRUE)
clust.1688 <- dada.1688$clustering[!bim.1688,]
cat("DADA2 identified", nrow(clust.1688), "non-chimeric sequence variants.\n")


One of the bonuses of DADA2 is that in selfConsist mode the algorithm is jointly inferring the error rates in your data as a function of the quality score. Take a look:


In [ ]:
plotErrors(dada.1688, nominalQ=TRUE)


The red line is the expected error rate if the quality scores were exactly right, the black line is DADA2's fitted error model. Notice how the observed error rates flatten out at higher qualities. This is typical, although has improved in more recent Illumina machines/software.

### Evaluating DADA2's output

This mock community is supposed to be a mixture of 48 bacterial strains (see Supplementary Table 6 in [the original Bokulich paper](http://dx.doi.org/10.1038/nmeth.2276)), which is fairly close to the 64 sequence variants output by DADA2 (and a long ways away from the 1,000+ OTUs produced by Swarm, SUMACLUST and uclust). However, accuracy at the OTU-level wasn't measured in the paper, the focus instead being on accuracy after grouping into broader taxonomic categories.

We'll follow the paper's approach and do the same here. We'll assign taxonomies using DADA2's assignTaxonomy function, which implements the RDP classifier method, and the GreenGenes 13.8 database clustered at 97% (*the DADA2 formatted training fasta file [is available here](https://www.dropbox.com/sh/atmpqqm9zxwf6qf/AAB9MPLGTh6ZWWBAV1HSWNoca?dl=0)*):


In [ ]:
set.seed(100)
tax.1688 <- assignTaxonomy(clust.1688$sequence, "ggtrain_97.fa", minBoot=50)
tax.1688 <- gsub("\\[", "", tax.1688)
tax.1688 <- gsub("\\]", "", tax.1688)
justGenus <- function(taxs) {
  rval <- character(0)
  for(i in seq_along(taxs)) {
    ranks <- strsplit(taxs[[i]], ";")[[1]]
    rval[[i]] <- paste(ranks[1:6], collapse=";")
  }
  rval <- gsub(";NA", "", rval)
  rval
}
genus.1688 <- justGenus(tax.1688)


From the original Bokulich paper (Supplementary Table 6), we know the the identities of the 48 strains intended to be in this mock community. We can extract the rRNA sequences from the known genomes of those strains, classify the sequenced region with RDP against GreenGenes, and thereby generate the taxonomies we expect to see in our data down to the genus level:


In [ ]:
# Expected taxonomies
exp.1688 <- c("k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Bifidobacteriales;f__Bifidobacteriaceae;g__Bifidobacterium",
"k__Bacteria;p__Actinobacteria;c__Coriobacteriia;o__Coriobacteriales;f__Coriobacteriaceae;g__Collinsella",
"k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides",
"k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Porphyromonadaceae;g__Parabacteroides",
"k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Rikenellaceae;g__Alistipes",
"k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus",
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Tissierellaceae;g__Anaerococcus", #
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Clostridiaceae;g__Clostridium",
"k__Bacteria;p__Firmicutes;c__Erysipelotrichi;o__Erysipelotrichales;f__Erysipelotrichaceae;g__Eubacterium", #
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae",
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Blautia",
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Clostridium",
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Dorea",
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Lachnospira", #
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Roseburia",
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Lachnospiraceae;g__Ruminococcus", #
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Anaerotruncus",
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Faecalibacterium",
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Ruminococcaceae;g__Ruminococcus",
"k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacteriales;f__Enterobacteriaceae;g__Edwardsiella",
"k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacteriales;f__Enterobacteriaceae", #
"k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacteriales;f__Enterobacteriaceae;g__Escherichia", #
"k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacteriales;f__Enterobacteriaceae;g__Pantoea",
"k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacteriales;f__Enterobacteriaceae;g__Proteus",
"k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacteriales;f__Enterobacteriaceae;g__Providencia",
"k__Bacteria;p__Verrucomicrobia;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Verrucomicrobiaceae;g__Akkermansia")


And then compare the output of DADA2 to those expectations:


In [ ]:
cat("DADA2 detected", sum(exp.1688 %in% genus.1688), "out of", length(exp.1688), "expected taxonomies.\n")
cat("Missing taxonomy:", exp.1688[!exp.1688 %in% genus.1688], "\n")
cat("Within the Bacteroides genus, DADA2 detected", sum(grepl("Bacteroides", tax.1688)), "sequence variants.")


DADA2 has detected essentially all the expected taxonomies. The one exception appears to be because no *Collinsella intestinalis* sequences made it into the dataset. Furthermore, DADA2 detected real variation at higher-resolution than the genus level. For example, there were 12 Bacteroides among the 48 strains combined in this community, and DADA2 detected 12 Bacteroides sequence variants.

Now taking a closer look at the "false positives", the sequences output by DADA2 that were not classified into an expected taxonomy:


In [ ]:
unexp.1688 <- !genus.1688 %in% exp.1688
cat("DADA2 detected", sum(unexp.1688), "sequence variants with unexpected taxonomies with abundances:", clust.1688$abundance[unexp.1688], "\n")
uniquesToFasta(clust.1688[unexp.1688,], "unexpected_1688.fa")
# BLAST against nt on NCBI website


Every one of the `r sum(unexp.1688)` sequences from the `r length(unique(genus.1688[unexp.1688]))` unexpected taxonomies is an exact (100\% coverage and 100\% identity) BLAST hit to nt. While unexpected, these are not false positives, they are correctly identified contaminantss.



In [ ]:
getF <- function(exp, genus) {
  TP <- sum(exp %in% genus)
  FP <- length(unique(genus[!genus %in% exp]))
  FN <- sum(!exp %in% genus)
  precision <- TP/(TP+FP)
  recall <- TP/(TP+FN)
  Fscore <- 2*precision*recall/(precision+recall)
  return(Fscore)
}


Following the accuracy methodology in the paper, DADA2 has an F-score of **`r getF(exp.1688, genus.1688)`** on this dataset, with all `r length(unique(genus.1688[unexp.1688]))` "false positives" being "known"

## Mock Community 1685

This mock community dataset (also Bokulich_2 in the paper) is a combination of 4 samples of the same mixture of 18 strains. It is the largest Bokulich dataset evaluated in this paper, at 6.9M 250nt reads sequences from a Miseq. DADA2 expects different samples to be demultiplexed into individual fastq files, but this dataset is treated as one big sample in the paper so we follow that lead and just process it all together.

### Filtering and Trimming

We again start by removing the phiX contamination:


In [ ]:
####### TO BE EXECUTED BY HAND IN YOUR SHELL
### awk '{ if (NR % 4 == 1) print ">",((NR+3)/4); else if (NR % 4 == 2) print $0; }' seqs.fastq > seqs.fasta
### blastn -query seqs.fasta -db phix -out phix_hits.out -max_target_seqs 1 -outfmt "6 qacc evalue"
### python rm_hits.py phix_hits.out seqs.fastq 1685_seqs_nophix.fastq
### gzip 1685_seqs_nophix.fastq


This reduces the number of sequences from 6,938,836 to 6,848,112. *You can also [download the phiX free fastq.gz file here](https://www.dropbox.com/sh/atmpqqm9zxwf6qf/AAB9MPLGTh6ZWWBAV1HSWNoca?dl=0)*.

Inspecting the quality profile of the data:


In [ ]:
fn.1685 <- "supplemental_otu_clustering_datasets/UPARSE_not_filtered_QIIME_label_format/16S/1685/1685_seqs_nophix.fastq.gz"
qq <- qa(fn.1685)[["perCycle"]]$quality
ShortRead:::.plotCycleQuality(qq, main="Bokulich 1685")


Quality scores drop sharply at the end of the sequences. We trim the last 20 bases and the first 10 bases (the first bases in Illumina sequencing are more likely to contain pathological errors):


In [ ]:
filt.1685 <- "filt_1685.fastq.gz"
if(file.exists(filt.1685)) file.remove(filt.1685)
fastqFilter(fn.1685, filt.1685, trimLeft=10, truncLen=230, truncQ=2, maxEE=2, maxN=0, verbose=TRUE)


### Running DADA2

Entering the DADA2 pipeline (*this section involves some reasonably serious computation, and takes about 1.5 hours to complete on my laptop*):


In [ ]:
derep.1685 <- derepFastq(filt.1685, verbose=TRUE)
dada.1685 <- dada(derep.1685, err=inflateErr(tperr1,3), selfConsist=TRUE)
bim.1685 <- isBimeraDenovo(dada.1685, verbose=TRUE)
clust.1685 <- dada.1685$clustering[!bim.1685,]
cat("DADA2 identified", nrow(clust.1685), "non-chimeric sequence variants.\n")
set.seed(100)
tax.1685 <- assignTaxonomy(clust.1685$sequence, "ggtrain_97.fa", minBoot=50)
genus.1685 <- justGenus(tax.1685)


### Evaluating DADA2's output

While DADA2 identified far fewer variants than any of the tested methods, it still found significantly more than the 18 variants expected. We'll return to that point, but for now we'll follow the same procedure of taxonomically classifying and comparing to expectations as is done in the paper.

First, the list of expected taxonomies (from classifying the type sequences of the strains listed in Bokulich Supplementary Table 4 against GreenGenes):


In [ ]:
exp.1685 <- c("k__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanobrevibacter",
"k__Bacteria;p__[Thermi];c__Deinococci;o__Deinococcales;f__Deinococcaceae;g__Deinococcus",
"k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces",
"k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Propionibacteriaceae;g__Propionibacterium",
"k__Bacteria;p__Bacteroidetes;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Bacteroides",
"k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Bacillaceae;g__Bacillus",
"k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Listeriaceae;g__Listeria",
"k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Staphylococcaceae;g__Staphylococcus",
"k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Enterococcaceae;g__Enterococcus",
"k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Lactobacillaceae;g__Lactobacillus",
"k__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus",
"k__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;f__Clostridiaceae;g__Clostridium",
"k__Bacteria;p__Proteobacteria;c__Alphaproteobacteria;o__Rhodobacterales;f__Rhodobacteraceae;g__Rhodobacter",
"k__Bacteria;p__Proteobacteria;c__Betaproteobacteria;o__Neisseriales;f__Neisseriaceae;g__Neisseria",
"k__Bacteria;p__Proteobacteria;c__Epsilonproteobacteria;o__Campylobacterales;f__Helicobacteraceae;g__Helicobacter",
"k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Enterobacteriales;f__Enterobacteriaceae;g__Escherichia",
"k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pseudomonadales;f__Moraxellaceae;g__Acinetobacter",
"k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Pseudomonadales;f__Pseudomonadaceae;g__Pseudomonas")


Now we compare the taxonomies assigned to the variants identified by DADA2 to these expected taxonomies:


In [ ]:
cat("DADA2 detected", sum(exp.1685 %in% genus.1685), "out of", length(exp.1685), "expected taxonomies.\n")
cat(sum(clust.1685$abundance[genus.1685 %in% exp.1685]), "reads out of", sum(clust.1685$abundance), "total reads were in the expected taxonomies.\n")


Over 99.99\% of total reads were assigned to the expected taxonomies. But taking a look at the remaining:


In [ ]:
unexp.1685 <- !genus.1685 %in% exp.1685
cat("DADA2 detected", sum(unexp.1685), "unexpected taxonomies with abundances:", clust.1685$abundance[unexp.1685], "\n")
print(genus.1685[unexp.1685])
uniquesToFasta(clust.1685[unexp.1685,], "unexpected_1685.fa")
# BLAST against nt on NCBI website


8 of the 11 sequences from unexpected taxonomies were exact BLAST matches, while the remaining 3 were within 99\% identity of previously sequenced strains. Once again, these aren't really false positives, they are correctly identified contaminants.

Following the accuracy methodology in the paper, DADA2 has an F-score of **`r getF(exp.1685, genus.1685)`** on this dataset, with all `r length(unique(genus.1685[unexp.1685]))` "false positives" being "known"

### DADA2 Diagnostics and Insanely Great Sample Inference

At the level of genera DADA2 very accurately reconstructed this mock community, but at the level of sequence variants the `r nrow(clust.1685)` sequences output by DADA2 is significantly above the 18 strains expected (although much closer to the truth than the 7,000+ OTUs output by many of the other tested method).

One of the nice things about DADA2 is that it provides some very useful diagnostics about each output sequence in the `dadaObject$clustering` data.frame. Inspecting the head of that data.frame (after removing chimeras):


In [ ]:
head(clust.1685)


The sequence and abundance columns are self-explanatory, but several other columns are very useful in detecting spurious output sequences. In particular:

* `$n0`: The number of reads exactly matching the output sequence.
* `$birth_fold`: The fold over-abundance of the read relative to expectations under the error model.
* `$birth_ham`: The hamming distance of this sequence from the cluster from which it was split-off.
* `$birth_qave`: The average quality-score at the differences that caused this cluster to be split-off.

Low values of n0 relative to total abundance, low values of the birth\_fold and birth\_qave, and an overabundance of very low (typically just 1) birth_ham entries can all indicate spurious sequences.

Noting that the `$clustering` data.frame is arranged in order of how confident DADA2 was in each sequence, we look at the bottom of the data.frame for clues on why we are getting too many output sequences:


In [ ]:
tail(clust.1685[,-1], n=10)
table(clust.1685$birth_ham)


This is clear indication that, for this dataset, DADA2 is not effectively discriminating real single-nucleotide variants from errors. There are various reasons why this happens, and [DADA2 provides effective single-nucleotide resolution](http://biorxiv.org/content/early/2015/08/06/024034) in many other datasets, but here it is not working.

So, we give up on having single-nucleotide resolution here, and re-cluster requiring a minimum hamming separation of 2 to call new sequence variants:


In [ ]:
dada.1685.noSNV <- dada(derep.1685, err=dada.1685$err_out, MIN_HAMMING=2)
bim.1685.noSNV <- isBimeraDenovo(dada.1685.noSNV, verbose=TRUE)
clust.1685.noSNV <- dada.1685.noSNV$clustering[!bim.1685.noSNV,]
cat("DADA2 identified", nrow(clust.1685.noSNV), "non-chimeric sequence variants.\n")
uniquesToFasta(clust.1685.noSNV, "denoised.1685.noSNV.fasta")
# BLAST against nt on NCBI website


This is what we expect, the 18 expected strains and handful of contaminants. Note that the 39 variants output by DADA2 here includes every expected strain, include rare variants present at frequences <0.001\%, and the sequences from unexpected genera almost all exactly match nt.

# Summary of Results

DADA2 was substantially more accurate than any of the methods tested in the paper, outperforming on both sensitivity and specificity.

### Simulated Data

For the even dataset, DADA2 detected **every one** of the 1,055 unique sequence variants, including those differing by just 1 nucleotide, more than any other method (957-1049). DADA2 reported 0 False Positives (amount unreported for other methods, but probably significant for some). DADA2's estimated abundances were essentially perfect, and just 5/106,700 sequencing reads were misassigned.

For the staggered dataset, DADA2 identified 1,042/1,055 unique sequence variants, more than any other tested method (949-1035). DADA2 again output 0 False Positives (amount unreported for other methods, but probably significant for some). DADA2's 13 False Negatives were all very low abundance (1-5 reads, <0.01% frequency). DADA2's estimated abundances were extremely accurate (cor > 0.9999).

Simulated data without chimeras is the easiest possible case, and performance on such data is not guaranteed to translate to real sequencing datasets. That said, none of the algorithms tested in this paper approached DADA2's level of accuracy. Tested at the "OTU" level, DADA2's F-score on the Even dataset is 1, and on the Staggered dataset is 0.994.

### Mock Communities

For the Bokulich 1688 (or Bokulich_6) dataset, DADA2 output 64 sequence variants, closer to the expected number of 48 strains than any other method (many of which reported >1000 OTUs). At the genus level, DADA2 identified 25/26 expected taxonomies, with the one missed taxonomy likely being absent from the sequencing data. DADA2 identified 6 unexpected genera, all of them low abundance (<0.5\%) and all of them exact BLAST matches to nt -- these are all likely correctly identified contaminants. DADA2's Fscore at the genus level was `r getF(exp.1688, genus.1688)`, higher than any other tested method (0.45 - 0.61).

For the Bokulich 1685 (or Bokulich_2) dataset, DADA2 output 175 sequence variants, closer to the expected number of 18 strains than any other method. At the genus level, DADA2 identified 18/18 expected taxonomies. DADA2 identified 11 unexpected genera, all of them extremely low abundance (<0.01\%) and all of them with a BLAST match of >99\% identity to nt -- these are all likely correctly identified contaminants. DADA2's Fscore at the genus level was `r getF(exp.1685, genus.1685)`, higher than any other tested method (0.39-0.54).

The default application of DADA2 was already more accurate than any of the other tested methods on Bokulich 1685, but with one extra step of using the provided diagnostics, and thereby deciding to not call single-nucleotide variants, DADA2 output 39 sequence variants, an almost error-free collection of the expected strains and contaminants. None of the algorithms tested in this paper approached this level of accuracy.

# Accuracy and DADA2

Common tools, including many of those tested in the paper [Open-Source Sequence Clustering Methods Improve the State Of the Art](http://msystems.asm.org/content/1/1/e00003-15), report thousands of OTUs from amplicon sequencing data performed on mock communities of tens of strains due to inadequate treatment of PCR and sequencing error. This biases estimates of diversity, lowers the power of subsequent analysis, prevents the effective resolution of low-frequency variants, and limits amplicon studies to characterizing shifts in broad taxonomic groups.

As shown above, our method DADA2 improves on both the sensitivity and specificity of commonly used methods, detecting both more real variants and (far) fewer spurious ones. This difference can be dramatic: The common QIIME/uclust pipeline reports 20,084 OTUs in an 18-strain mock community, ~20,000 of which are spurious. In comparison, DADA2 without SNVs reports just 39 sequence variants, which consist of the 18 expected strains, a handful of contaminants, and ~5 spurious sequences. Of note, this dramatic reduction in spurious output is achieved without imposing aggressive filters.

Accurate analyses start with accurate data, and [DADA2](http://benjjneb.github.io/dada2/) improves the accuracy of amplicon sequencing data by powerfully separating sequence errors from true biological diversity.
